In [ ]:
! kaggle competitions download multilingualabusivecomment

 96% 48.0M/50.0M [00:00<00:00, 81.5MB/s]
100% 50.0M/50.0M [00:00<00:00, 113MB/s] 


In [ ]:
! unzip /content/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv.zip

Archive:  /content/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv.zip
  inflating: ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv  


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv',error_bad_lines=False)

In [ ]:
data.head()

,CommentId,commentText,language,user_index,post_index,report_count_comment,report_count_post,like_count_comment,like_count_post,label
0,0,randi ka deewana,Assamese,2869,175913,0,1,0,65,1.0
1,1,👌👌🙏🥰🥰🥰☝️☝️,Rajasthani,601,960582,0,0,1,94,0.0
2,2,apni pant utar kar apni chut bhi to bata do ma...,Urdu,1205,17327,0,1,0,731,1.0
3,3,krishnatripathi4578 jesi ghar ki sabhyata vais...,Rajasthani,889,352919,0,1,1,285,0.0
4,4,is randi ko dusra kaam nahi hai kuttiya kahin ki,Urdu,372,261223,0,0,0,453,1.0


In [ ]:
len(data)

1483506

In [ ]:
data.columns

Index(['CommentId', 'commentText', 'language', 'user_index', 'post_index',
       'report_count_comment', 'report_count_post', 'like_count_comment',
       'like_count_post', 'label'],
      dtype='object')

In [ ]:
docs = list(data['commentText'].values)
target = list(data['label'])

In [ ]:
from keras.preprocessing.text import one_hot
vocab_size = 1183500
encoded_docs = []
target_cleaned = []
for i in range(len(docs)):    
  try:
    encoded_docs.append(one_hot(docs[i], vocab_size))
    target_cleaned.append(target[i])
  except:
    print(docs[i])

nan


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(encoded_docs, target_cleaned, test_size=0.2)

In [ ]:
words = []
for sentences in docs:
  try:
    words.extend(sentences.split(' '))
  except:
    print(sentences)

nan


In [ ]:
len(encoded_docs), len(target_cleaned)

(1483505, 1483505)

In [ ]:
from keras.preprocessing import sequence 
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length) 
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length) 


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Convolution1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import numpy as np


In [ ]:
embedding_vector_length = 32 
model = Sequential() 
model.add(Embedding(max_review_length, embedding_vector_length, input_length=max_review_length)) 
model.add(LSTM(100)) 
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           16000     
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 69,301
Trainable params: 69,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train, np.array(y_train), validation_data=(X_test, np.array(y_test)), epochs=1, batch_size=64) 


18544/18544 [==============================] - 1543s 83ms/step - loss: nan - accuracy: 0.7654 - val_loss: nan - val_accuracy: 0.7667


In [ ]:
scores = model.evaluate(X_test, np.array(y_test), verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 76.67%
